In [26]:
%%time

import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from libpysal.graph import read_parquet
from sklearn.preprocessing import StandardScaler

from fast_hdbscan.numba_kdtree import kdtree_to_numba
from fast_hdbscan.numba_kdtree import parallel_tree_query
from sklearn.neighbors import KDTree

CPU times: user 16 μs, sys: 0 ns, total: 16 μs
Wall time: 16.9 μs


In [27]:
region_id = 69300

tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
chars_dir = "/data/uscuni-ulce/processed_data/chars/"
graph_dir = "/data/uscuni-ulce/processed_data/neigh_graphs/"
val_path = '../data/prague_clusters.parquet'

In [28]:
region_id = 'freiburg'
buildings_dir = streets_dir = enclosures_dir = tessellations_dir = graph_dir = '../data/freiburg/'
chars_dir = '../data/freiburg/chars/'
val_path = '../data/fbg_cluster_validation.pq'

In [29]:
graph = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")

In [30]:
graph.cardinalities.describe()

count    36727.000000
mean         6.897841
std          2.126293
min          1.000000
25%          6.000000
50%          7.000000
75%          8.000000
max         65.000000
Name: cardinalities, dtype: float64

In [31]:
from core.cluster_validation import print_distance, generate_neigbhourhood_groups, generate_detailed_clusters

In [117]:
tessellation = gpd.read_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
)

In [33]:
X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')



X_train = X_train[X_train.index >= 0]



spatial_lag = 3


# lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/context_chars_{region_id}_lag_{spatial_lag}.parquet')

lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/unprocessed_context_chars_{region_id}_lag_{spatial_lag}.parquet')



X_train = X_train.join(lag[[c for c in lag.columns if '_median' in c]], how='inner')

# X_train = X_train.join(lag, how='inner')


In [34]:
# for c in X_train.columns:
#     X_train[c] = X_train[c].clip(*np.percentile(X_train[c], [5, 95]))

In [35]:
to_drop = ['stcSAl',
 'ltkOri',
 'stbOri',
 'stcOri',
 'stbCeA',

#not in barcelona
 # 'ltcBuA', 'midRea', 'midAre', 'likWBB'
]


all_drop = []
for c in to_drop:
    all_drop += X_train.columns[X_train.columns.str.contains(c)].tolist()


X_train = X_train.drop(all_drop, axis=1)

In [36]:

# vals = StandardScaler().fit_transform(X_train)
# X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)

# vals = np.nan_to_num(X_train)
# X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)

In [37]:
# t1 = X_train[[c for c in X_train.columns if '_' not in c]]
# X_train = X_train[[c for c in X_train.columns if '_median' in c]]

# X_train = t1.join(t2)
# X_train.shape

In [38]:
# X_train = X_train.drop(stats.columns[stats.loc['std'] == 0], axis=1)

In [39]:
X_train.shape

(36067, 116)

In [118]:
tess_groups = generate_detailed_clusters(tessellation,
                                         include_random_sample=False, path=val_path)
tess_groups = tess_groups[tess_groups.index.isin(X_train.index)]
tess_groups_ilocs = (
    pd.Series(np.arange(len(X_train)), index=X_train.index)
    .loc[tess_groups.index]
    .values
)

from sklearn.metrics import davies_bouldin_score

def check_score(data, example_clusters):
    groups = example_clusters[example_clusters.index.isin(data.index)]
    groups_ilocs = (
        pd.Series(np.arange(len(data)), index=data.index).loc[groups.index].values
    )
    return davies_bouldin_score(data.iloc[groups_ilocs], groups.values)

# check_score(X_train, tess_groups)

In [84]:
# tessellation.loc[tess_groups.index].explore()

In [85]:
from core.cluster_validation import print_distance
print_distance( pd.DataFrame(X_train.loc[tess_groups.index]).groupby(tess_groups.values).mean(), metric='euclidean')

,Haslach industrial,altstadt,haid industrial,haslach multi,herdern quiet,industrial north,lorettoberg,oberau,rieselfeld multi,riselfeld single,seepark single famil,stÃ¼hlinger,vauban,weingarten multifami
Haslach industrial,0.000000,324070.519909,297101.479927,290819.022676,372097.134995,449680.770113,103081.905812,292672.600343,286484.834884,203996.544559,271072.498978,283085.845284,117023.986662,265594.911341
altstadt,324070.519909,0.000000,467971.632700,89429.745632,486665.779135,722124.429173,389572.265367,430410.471566,103537.172385,261788.476132,200265.370851,119549.592532,260470.329632,553046.384457
haid industrial,297101.479927,467971.632700,0.000000,387128.516927,98463.415466,297901.778015,212374.404021,71148.856481,373360.359789,485324.375956,309260.686991,361054.761125,369594.736042,493580.275912
haslach multi,290819.022676,89429.745632,387128.516927,0.000000,399895.220470,652258.466362,336537.692806,346967.571400,22943.896003,288974.964908,122331.019088,31853.433338,245731.895257,542272.505489
herdern quiet,372097.134995,486665.779135,98463.415466,399895.220470,0.000000,352608.986545,295800.939972,103361.764064,386360.739847,546536.778138,314599.242489,372211.942387,427469.865030,583086.891254
industrial north,449680.770113,722124.429173,297901.778015,652258.466362,352608.986545,0.000000,352035.846057,343355.673962,641432.787928,648403.698004,568732.068322,628663.509058,537889.969560,491306.701404
lorettoberg,103081.905812,389572.265367,212374.404021,336537.692806,295800.939972,352035.846057,0.000000,217542.424779,328987.397247,304082.147283,285646.027222,320453.662726,198421.116824,294354.069359
oberau,292672.600343,430410.471566,71148.856481,346967.571400,103361.764064,343355.673962,217542.424779,0.000000,334811.908170,466629.049939,263081.008730,318827.859353,349764.784201,507004.155359
rieselfeld multi,286484.834884,103537.172385,373360.359789,22943.896003,386360.739847,641432.787928,328987.397247,334811.908170,0.000000,295460.559978,121521.565249,30686.519870,249359.304163,541042.607595
riselfeld single,203996.544559,261788.476132,485324.375956,288974.964908,546536.778138,648403.698004,304082.147283,466629.049939,295460.559978,0.000000,329092.128856,300540.700898,140676.092975,322098.163248


In [86]:
from scipy.spatial.distance import pdist, cdist
for i, g in X_train.loc[tess_groups.index].groupby(tess_groups.values):
    print(i, np.mean(pdist(g)))

Haslach industrial nan
altstadt nan
haid industrial nan
haslach multi nan
herdern quiet nan
industrial north nan
lorettoberg nan
oberau nan
rieselfeld multi nan
riselfeld single nan
seepark single famil nan
stÃ¼hlinger 80687.1812755953
vauban nan
weingarten multifami nan


In [87]:
# tessellation.loc[tess_groups.index].explore(column=tess_groups.values, categorical=True)

In [89]:
# np.min(cdist(X_train.loc[tess_groups[tess_groups == 'josefov'].index], 
#              X_train.loc[tess_groups[tess_groups == 'stare mesto'].index]))

In [22]:
mean_clusters = pd.DataFrame(X_train.loc[tess_groups.index]).groupby(tess_groups.values).mean()

In [23]:
(mean_clusters.loc['josefov'] - mean_clusters.loc['stare mesto']).abs().sort_values(ascending=False).iloc[:10]

ldsAre           20064.969718
ldkAre_median    19555.315873
ldsAre_median    19472.008560
ldkAre           18241.150667
sdsAre            1836.346662
sdsAre_median     1499.228325
sddAre            1030.843759
mdcAre             997.143153
sddAre_median      895.913718
midAre             795.110295
dtype: float64

In [24]:
from core.utils import used_keys
used_keys['lcnClo']

'local closeness of street network'

In [25]:
# training_data = X_train.loc[tess_groups.index]
# tess_groups_ilocs = (
#     pd.Series(np.arange(len(training_data)), index=training_data.index)
#     .loc[tess_groups.index]
#     .values
# )
# training_data.shape

In [90]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import davies_bouldin_score
from core.cluster_validation import get_linkage_matrix

In [97]:
q1 = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")

# clustering_graph = q1.higher_order(k=3, lower_order=True, diagonal=True).subgraph(X_train.index.values)

clustering_graph = q1.copy()

In [98]:
graph_labels = q1.subgraph(X_train.index.values).component_labels
graph_labels.value_counts()

component labels
3     28927
8      2878
11     1350
7      1060
4       771
25      614
14      186
27       79
28       60
29       43
12       18
9        17
30       16
6         8
26        7
23        5
10        4
32        3
33        2
5         2
35        2
31        1
24        1
34        1
0         1
22        1
21        1
20        1
19        1
1         1
17        1
16        1
15        1
13        1
2         1
18        1
Name: count, dtype: int64

In [99]:
clustering_graph = clustering_graph.subgraph(graph_labels[graph_labels == 3].index.values)
core_ids = clustering_graph.unique_ids


In [100]:
# clustering_graph = q1.subgraph(q1.component_labels[q1.component_labels == 25].index.values)
# clustering_graph = clustering_graph.subgraph(clustering_graph.unique_ids[clustering_graph.unique_ids >= 0])
# core_ids = clustering_graph.unique_ids

In [101]:
training_data = X_train[X_train.index.isin(core_ids)]

In [102]:


# training_data = training_data[[c for c in training_data.columns if '_' not in c]]


In [103]:
# t1 = training_data[[c for c in training_data.columns if '_' not in c]]
# t2 = training_data[[c for c in training_data.columns if '_median' in c]]

# training_data = t1.join(t2)

In [104]:
# training_data = X_train[X_train.index >=0]
# clustering_graph = graph.subgraph(X_train.index)

In [105]:
training_data.shape

(28927, 116)

In [106]:
# vals = StandardScaler().fit_transform(training_data)
# training_data = pd.DataFrame(vals, columns=training_data.columns, index=training_data.index)

vals = np.nan_to_num(training_data)
training_data = pd.DataFrame(vals, columns=training_data.columns, index=training_data.index)

stats = training_data.describe()
training_data = training_data.drop(stats.columns[stats.loc['std'] == 0], axis=1)

In [127]:
%%time
clusterer = AgglomerativeClustering(
    linkage='ward',
    # metric='cityblock',
                                                  
    connectivity = clustering_graph.transform('B').sparse, 
    
                                    # connectivity=q1.subgraph(X_train.index.values).transform('B').sparse, 
                                    compute_full_tree=True,
                                    compute_distances=True)
model = clusterer.fit(training_data)

CPU times: user 636 ms, sys: 0 ns, total: 636 ms
Wall time: 635 ms


In [128]:
linkage_matrix = get_linkage_matrix(model)

In [129]:
# fix, ax = plt.subplots(figsize=(40,40))
# # Plot the corresponding dendrogram
# _ = dendrogram(linkage_matrix, truncate_mode="level", p=5, ax=ax)

In [130]:
from sklearn.metrics import calinski_harabasz_score

In [131]:
tess_groups = generate_detailed_clusters(tessellation,
                                         include_random_sample=False, path=val_path)
tess_groups = tess_groups[tess_groups.index.isin(training_data.index)]
tess_groups_ilocs = (
    pd.Series(np.arange(len(training_data)), index=training_data.index)
    .loc[tess_groups.index]
    .values
)

In [132]:
first = linkage_matrix[0, 2]
last = linkage_matrix[-1, 2]
step = (last - first) / 25

In [133]:
res = []
i = 0
for i in range(0, 25):

    t = first + step *  (i + 1)
    if t >= linkage_matrix[-1, 2]:
        break
    
    r = fcluster(linkage_matrix, t=t, criterion='distance')
    r = pd.Series(r, index=training_data.index)
    ssplits = clustering_graph.describe(r, statistics=['nunique'])['nunique']

    res.append((t,
          adjusted_rand_score(tess_groups.values, r.iloc[tess_groups_ilocs]),
          (ssplits > 1).sum() / ssplits.shape[0],
          davies_bouldin_score(training_data, r),
          calinski_harabasz_score(training_data, r)
         ))
pd.DataFrame(res, columns = ['cutoff', 'rand', 'ssplits', 'db_score', 'ch_score']).set_index('cutoff')

,rand,ssplits,db_score,ch_score
cutoff,,,,
8.950294e+05,0.336260,0.429460,3.162499,1051.060329
1.790059e+06,0.468966,0.300653,4.231012,1357.576953
2.685088e+06,0.563768,0.243198,4.616638,1641.283192
3.580117e+06,0.588840,0.215612,5.526252,1860.132339
4.475146e+06,0.664947,0.185709,4.958248,2064.309926
5.370175e+06,0.743479,0.152764,4.508412,2218.479123
6.265204e+06,0.746481,0.150344,4.802263,2278.821738
7.160233e+06,0.740672,0.143430,4.706114,2307.673580
8.055262e+06,0.715736,0.124106,4.359553,2319.099853


In [123]:
# res = []
# for t in range(5, 25, 1):

#     if t >= linkage_matrix[-1, 2]:
#         break
    
#     r = fcluster(linkage_matrix, t=t, criterion='distance')
#     r = pd.Series(r, index=training_data.index)
#     ssplits = clustering_graph.describe(r, statistics=['nunique'])['nunique']

#     res.append((t,
#           adjusted_rand_score(tess_groups.values, r.iloc[tess_groups_ilocs]),
#           (ssplits > 1).sum() / ssplits.shape[0],
#           davies_bouldin_score(training_data, r),
#           calinski_harabasz_score(training_data, r)
#          ))
# pd.DataFrame(res, columns = ['cutoff', 'rand', 'ssplits', 'db_score', 'ch_score']).set_index('cutoff')

In [125]:
# try hdbscan extraction
from fast_hdbscan.boruvka import parallel_boruvka
from fast_hdbscan.cluster_trees import (
    cluster_tree_from_condensed_tree,
    condense_tree,
    extract_eom_clusters,
    get_cluster_label_vector,
    mst_to_linkage_tree,
)
from fast_hdbscan.numba_kdtree import kdtree_to_numba
from sklearn.neighbors import KDTree

In [126]:
res = []

for min_cluster_size in range(25, 500, 25):


    condensed_tree = condense_tree(linkage_matrix, 
                                   min_cluster_size=min_cluster_size)
    cluster_tree = cluster_tree_from_condensed_tree(condensed_tree)
    selected_clusters = extract_eom_clusters(
        condensed_tree, cluster_tree, allow_single_cluster=False
    )
    r = get_cluster_label_vector(condensed_tree, selected_clusters, 0)


    r = pd.Series(r, index=training_data.index)
    ssplits = clustering_graph.describe(r, statistics=['nunique'])['nunique']

    res.append((min_cluster_size,
          adjusted_rand_score(tess_groups.values, r.iloc[tess_groups_ilocs]),
          (ssplits > 1).sum() / ssplits.shape[0],
          davies_bouldin_score(training_data, r),
          calinski_harabasz_score(training_data, r)
         ))

pd.DataFrame(res, columns = ['min__cluster_size', 'rand', 'ssplits', 'db_score', 'ch_score']).set_index('min__cluster_size')

,rand,ssplits,db_score,ch_score
min__cluster_size,,,,
25,0.131810,0.629239,3.271012,334.155992
50,0.220492,0.490580,4.227782,489.483436
75,0.285869,0.416462,4.617014,635.205718
100,0.366029,0.370000,5.110760,742.430746
125,0.476657,0.331766,5.644523,923.887524
150,0.502507,0.307014,6.157895,1059.306729
175,0.512333,0.292218,6.439294,1121.997667
200,0.629567,0.278321,6.376744,970.032630
225,0.623845,0.262419,6.220068,928.740360


In [111]:


condensed_tree = condense_tree(linkage_matrix, 
                               min_cluster_size=150)
cluster_tree = cluster_tree_from_condensed_tree(condensed_tree)
selected_clusters = extract_eom_clusters(
    condensed_tree, cluster_tree, allow_single_cluster=False
)
clusters = get_cluster_label_vector(condensed_tree, selected_clusters, 0)

In [112]:
plotting = tessellation.loc[training_data.index].reset_index()

In [113]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 1.89 s, sys: 80.6 ms, total: 1.98 s
Wall time: 1.23 s


In [114]:
from sidecar import Sidecar
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [115]:
from core.cluster_validation import get_color

In [124]:
clusters = fcluster(linkage_matrix, t=589011.8, criterion='distance')

In [120]:


condensed_tree = condense_tree(linkage_matrix, 
                               min_cluster_size=200)
cluster_tree = cluster_tree_from_condensed_tree(condensed_tree)
selected_clusters = extract_eom_clusters(
    condensed_tree, cluster_tree, allow_single_cluster=False
)
clusters = get_cluster_label_vector(condensed_tree, selected_clusters, 0)

In [121]:
# np.unique(clusters, return_counts=True)

In [125]:
layer.get_fill_color = get_color(clusters)

In [100]:
new_data = training_data.groupby(clusters).mean()

,rand,ssplits,db_score,ch_score
min__cluster_size,,,,
25,0.190729,0.622261,3.223599,51.622499
50,0.408159,0.469656,3.772883,81.835517
75,0.467800,0.384873,4.168669,105.134981
100,0.515796,0.349137,4.393777,121.666307
125,0.562180,0.306911,4.584831,150.763636
150,0.668354,0.266039,4.843566,176.639812
175,0.653469,0.248534,5.076343,200.022951
200,0.654747,0.237272,5.244260,218.248270
225,0.654832,0.227925,5.344687,229.874044


In [157]:
from core.cluster_validation import get_feature_importance
from core.utils import used_keys

In [69]:
clusters_subset = [11597, 11615, 17742]
clusters_subset = np.where(np.isin(clusters, clusters[clusters_subset]))

In [70]:
imps = get_feature_importance(training_data.iloc[clusters_subset], clusters[clusters_subset])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [72]:
imps.loc[:10, [c for c in imps.columns if '_vals' not in c]]

,cluster_48,cluster_66,cluster_69
0,linWID_higher,lcdMes_higher,ssbCCD_lower
1,ldsMSL_lower,linPDE,ltkOri_lower
2,lddNDe,linP3W_higher,ssbSqu_lower
3,ltkOri_lower,mtbAli_median,lcdMes_median
4,ldsMSL,lskCWA_lower,linWID_higher
5,ldsMSL_median,lskERI_median,sicCAR_lower
6,midRea_median,ltkWNB_median,lskCWA_lower
7,ldkAre_higher,linWID_lower,stcOri_lower
8,lskCCo_lower,mtbNDi_higher,ssbSqu_median
9,ssbCCD_lower,linPDE_lower,sdcLAL_median


In [77]:
used_keys['ltkOri']

'orientation of enclosure'

In [112]:
imps = get_feature_importance(training_data, clusters)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 co

In [119]:
imps.loc[:10, [c for c in imps.columns if '_vals' not in c]]

,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
0,ldkPer,lcdMes,libNCo,lskCWA,ldsAre,ltkWNB,likWBB
1,ltkWNB,likWBB,likWBB,likWBB,midAre,ldkPer,lcdMes
2,lskCCo,ltkWNB,sdsSPO,ltkWNB,ltkOri,lskCCo,ltkWNB
3,ltkOri,lskERI,ldbPWL,lskCCo,linP4W,lskCWA,linWID
4,lskCWA,lskCWA,ltcBuA,ldkAre,sdsSPO,ltkOri,ltkOri
5,ldkAre,ldkAre,sicCAR,lskERI,ldsMSL,likWBB,lskCWA
6,likWBB,linP4W,mtbSWR,ldsAre,lddNDe,ldkAre,ldkAre
7,lcdMes,ldkPer,ssbSqu,lddNDe,lcdMes,lskERI,lddNDe
8,lskERI,ltkOri,lskERI,midRea,ldkPer,linP3W,linP3W
9,linP4W,linPDE,sdbPer,linPDE,likWBB,ldsAre,lcnClo


In [122]:
used_keys['lcdMes']

'local meshedness of street network'

In [ ]:
imps[[c for c in imps.columns if '_vals' in c]].cumsum(axis=1)

In [96]:
josefov_joins = []

josefov_joins.append(np.isin(linkage_matrix[:, 0], 
                             tess_groups_ilocs[tess_groups == 'josefov']))
josefov_joins.append(np.isin(linkage_matrix[:, 1], 
                             tess_groups_ilocs[tess_groups == 'josefov']))


In [97]:
indxs = linkage_matrix[josefov_joins[0] | josefov_joins[1]]
indxs = np.union1d(indxs[:, 0], indxs[:, 1])
indxs = indxs[indxs <= X_train.shape[0]]

In [150]:
indxs = linkage_matrix[linkage_matrix[:, 2] <= 2]
indxs = np.union1d(indxs[:, 0], indxs[:, 1])
indxs = indxs[indxs < X_train.shape[0]]
indxs.shape

(64349,)

In [151]:
plotting = tessellation.loc[X_train.iloc[indxs].index]

In [42]:
cluster_means = training_data.groupby(clusters).mean()

In [68]:
c1 = 6
c2 = 10

(cluster_means.loc[c1] - cluster_means.loc[c2]).sort_values(ascending=False)

libNCo           6.284668
libNCo_median    5.846288
libNCo_higher    4.233946
linPDE_higher    3.014914
linPDE           2.122206
                   ...   
lcnClo          -1.414220
linWID_lower    -1.422183
linP3W          -1.489277
lcnClo_lower    -1.702326
linP3W_lower    -2.339139
Length: 248, dtype: float64

In [72]:
from core.utils import used_keys
used_keys['libNCo']

'number of courtyards within adjacent buildings'

In [169]:
bgraph = read_parquet(graph_dir + f"building_graph_{region_id}_knn1.parquet")

In [170]:
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'

buildings = gpd.read_parquet(
        buildings_dir + f"buildings_{region_id}.parquet"

)

In [178]:
buildings

,index,id,geometry
0,0,v0.1-CZE.12.2_1-35164,"POLYGON ((4614847.626 2975218.938, 4614848.235..."
1,1,v0.1-CZE.12.2_1-35123,"POLYGON ((4615276.357 2976034.184, 4615282.866..."
2,2,v0.1-CZE.12.2_1-35159,"POLYGON ((4615315.503 2975986.2, 4615322.056 2..."
3,3,v0.1-CZE.12.2_1-35166,"POLYGON ((4615222.339 2976016.91, 4615224.582 ..."
4,4,v0.1-CZE.12.2_1-35228,"POLYGON ((4615300.348 2975924.258, 4615301.6 2..."
...,...,...,...
299059,299060,v0.1-CZE.13.3_1-13696,"POLYGON ((4618611.169 3033535.197, 4618623.01 ..."
299060,299061,v0.1-CZE.13.3_1-13674,"POLYGON ((4618611.989 3033568.153, 4618617.119..."
299061,299062,v0.1-CZE.13.3_1-13591,"POLYGON ((4618614.831 3033550.704, 4618628.289..."
299062,299063,v0.1-CZE.13.3_1-13328,"POLYGON ((4618625.628 3033512.926, 4618625.634..."


In [181]:
buildings = buildings.join(X_train, how='inner').drop(['index', 'id'], axis=1)

In [185]:
r = buildings.dissolve(bgraph.component_labels, aggfunc='mean')

In [189]:
plotting = r

In [190]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 1.65 s, sys: 178 ms, total: 1.83 s
Wall time: 1.82 s


In [191]:
from sidecar import Sidecar
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [156]:
enclosures = gpd.read_parquet(f"/data/uscuni-ulce/processed_data/enclosures/enclosure_{region_id}.parquet")
encl_counts = tessellation.groupby('enclosure_index').count()
encl_counts.columns = ['tessellation']
enclosures['lieWCe'] = encl_counts['tessellation'] / enclosures.geometry.area

7        0.000325
8        0.000224
11       0.000030
12       0.000122
14       0.000638
           ...   
25157    0.000100
25158    0.000029
25159    0.000101
25160    0.000399
25161    0.000081
Length: 15958, dtype: float64

In [251]:
enclosures['lieWCe'] 

7        0.000325
8        0.000224
11       0.000030
12       0.000122
14       0.000638
           ...   
25157    0.000100
25158    0.000029
25159    0.000101
25160    0.000399
25161    0.000081
Name: lieWCe, Length: 15958, dtype: float64

In [164]:
encl_counts['tessellation']

enclosure_index
7        199
8         52
11         1
12        25
14       962
        ... 
25157      9
25158      1
25159     13
25160     15
25161      3
Name: tessellation, Length: 15935, dtype: int64

In [ ]:
# Measure weighted cells within enclosure
merged = enclosures[['eID', 'ldeAre']].merge(encl_counts[['tessellation']], how='left', on='eID')
enclosures['lieWCe'] = merged['tessellation'] / merged['ldeAre']

In [1]:
import geopandas as gpd

In [11]:
gpd.read_parquet('/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_0.pq').head()

,gml_id,identifier,oid,aktualit,gebnutzbez,funktion,gfkzshh,gmdschl,lagebeztxt,name,rellage,geometry
0,DEMVAL040000ACAEBL,urn:adv:oid:DEMVAL040000ACAEBL,DEMVAL040000ACAEBL,2014-11-04Z,Gebäude,Wohngebäude mit Handel und Dienstleistungen,31001_1120,13004000,None,None,None,"MULTIPOLYGON (((33263213.163 5948562.945, 3326..."
1,DEMVAL040000ACAdBL,urn:adv:oid:DEMVAL040000ACAdBL,DEMVAL040000ACAdBL,2014-11-04Z,Gebäude,Wohngebäude mit Handel und Dienstleistungen,31001_1120,13004000,Lindenstr. 7,None,None,"MULTIPOLYGON (((33263198.96 5948568.636, 33263..."
2,DEMVAL040000ACAjBL,urn:adv:oid:DEMVAL040000ACAjBL,DEMVAL040000ACAjBL,2014-11-04Z,Gebäude,Schuppen,31001_2723,13004000,None,None,None,"MULTIPOLYGON (((33263207.634 5948544.851, 3326..."
3,DEMVAL040000ACApBL,urn:adv:oid:DEMVAL040000ACApBL,DEMVAL040000ACApBL,2014-11-04Z,Gebäude,Garage,31001_2463,13004000,None,None,None,"MULTIPOLYGON (((33263213.625 5948581.535, 3326..."
4,DEMVAL040000ACAuBL,urn:adv:oid:DEMVAL040000ACAuBL,DEMVAL040000ACAuBL,2014-11-04Z,Gebäude,Schuppen,31001_2723,13004000,None,None,None,"MULTIPOLYGON (((33263212.119 5948542.451, 3326..."


In [12]:
gpd.read_parquet('/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_10000.pq').head()

,gml_id,identifier,oid,aktualit,gebnutzbez,funktion,gfkzshh,gmdschl,lagebeztxt,name,rellage,geometry
0,DEMVAL04000dfzxjBL,urn:adv:oid:DEMVAL04000dfzxjBL,DEMVAL04000dfzxjBL,2014-11-04Z,Gebäude,Garage,31001_2463,13004000,None,None,None,"MULTIPOLYGON (((33262346.344 5951813.908, 3326..."
1,DEMVAL04000dfzxpBL,urn:adv:oid:DEMVAL04000dfzxpBL,DEMVAL04000dfzxpBL,2014-11-04Z,Gebäude,Garage,31001_2463,13004000,None,None,None,"MULTIPOLYGON (((33262350.432 5951814.077, 3326..."
2,DEMVAL04000dfzxuBL,urn:adv:oid:DEMVAL04000dfzxuBL,DEMVAL04000dfzxuBL,2014-11-04Z,Gebäude,Wohngebäude,31001_1000,13004000,"Seitenweg 5, 6",None,None,"MULTIPOLYGON (((33262357.645 5951805.808, 3326..."
3,DEMVAL04000dfzxzBL,urn:adv:oid:DEMVAL04000dfzxzBL,DEMVAL04000dfzxzBL,2014-11-04Z,Gebäude,Gebäude für Wirtschaft oder Gewerbe,31001_2000,13004000,None,None,None,"MULTIPOLYGON (((33262377.065 5951815.087, 3326..."
4,DEMVAL04000dfzyCBL,urn:adv:oid:DEMVAL04000dfzyCBL,DEMVAL04000dfzyCBL,2014-11-04Z,Gebäude,Gebäude für Wirtschaft oder Gewerbe,31001_2000,13004000,None,None,None,"MULTIPOLYGON (((33262421.737 5951538.959, 3326..."


In [13]:
gpd.read_parquet('/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_20000.pq').head()

,gml_id,identifier,oid,aktualit,gebnutzbez,funktion,gfkzshh,gmdschl,lagebeztxt,anzahlgs,rellage,name,geometry
0,DEMVAL040000fCpABL,urn:adv:oid:DEMVAL040000fCpABL,DEMVAL040000fCpABL,2014-11-04Z,Sonstiges Bauwerk oder sonstige Einrichtung,Carport,51009_1611,13004000,None,NaN,None,None,"MULTIPOLYGON (((33257489.011 5951551.659, 3325..."
1,DEMVAL040000fCpDBL,urn:adv:oid:DEMVAL040000fCpDBL,DEMVAL040000fCpDBL,2014-11-04Z,Sonstiges Bauwerk oder sonstige Einrichtung,Carport,51009_1611,13004000,None,NaN,None,None,"MULTIPOLYGON (((33257499.872 5951546.253, 3325..."
2,DEMVAL040000fCpGBL,urn:adv:oid:DEMVAL040000fCpGBL,DEMVAL040000fCpGBL,2014-11-04Z,Sonstiges Bauwerk oder sonstige Einrichtung,Carport,51009_1611,13004000,None,NaN,None,None,"MULTIPOLYGON (((33257516.544 5951540.715, 3325..."
3,DEMVAL040000fCpaBL,urn:adv:oid:DEMVAL040000fCpaBL,DEMVAL040000fCpaBL,2014-11-04Z,Sonstiges Bauwerk oder sonstige Einrichtung,Carport,51009_1611,13004000,None,NaN,None,None,"MULTIPOLYGON (((33258550.944 5951297.674, 3325..."
4,DEMVAL040000fCpdBL,urn:adv:oid:DEMVAL040000fCpdBL,DEMVAL040000fCpdBL,2014-11-04Z,Sonstiges Bauwerk oder sonstige Einrichtung,Carport,51009_1611,13004000,None,NaN,None,None,"MULTIPOLYGON (((33258609.477 5951444.515, 3325..."
